In [1]:
import builder
import ItemCBFKNNRecommender as calc_sim
import hybridRecommender
from hybridRecommender import Recommender
import scipy.sparse
import imp
imp.reload(builder)
imp.reload(calc_sim)
imp.reload(hybridRecommender)

<module 'hybridRecommender' from '/home/strenuus/dev/recommender-systems/delivery/hybridRecommender.cpython-36m-x86_64-linux-gnu.so'>

In [2]:
def start(local, test):
    b = builder.Builder(local, test)
    ICM = b.get_ICM(0.6)
    URM_train = b.get_URM_train()
    URM_train_transpose = b.get_URM_transpose_train()
    if local == True:
        URM_test = b.get_URM_test()
        return b, ICM, URM_train, URM_train_transpose, URM_test
    return b, ICM, URM_train, URM_train_transpose

In [3]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
def compute_similarity():
    slimSimilarity = scipy.sparse.load_npz('input/similarity/slimSIMOnline.npz')
    recommender = calc_sim.ItemCBFKNNRecommender(URM_train, ICM)
    contentSimilarity = recommender.fit(shrink=0.0, topK=12)
    recommender = calc_sim.ItemCBFKNNRecommender(URM_train, URM_train_transpose)
    collaborativeSimilarity = recommender.fit(shrink=1.0, topK=200)
    recommender = RP3betaRecommender(URM_train)
    recommender.fit(alpha=0.8, beta=0.3, topK=100)
    graphSimilarity = recommender.W_sparse
    recommender = calc_sim.ItemCBFKNNRecommender(URM_train, URM_train)
    userbasedSimilarity = recommender.fit(shrink=5.0, topK=100)
    return contentSimilarity, collaborativeSimilarity, userbasedSimilarity, slimSimilarity, graphSimilarity

def run_online(a, a1, a2, a3, a4):
    print_to_csv(contentSimilarity, collaborativeSimilarity, userbasedSimilarity, slimSimilarity, graphSimilarity, a, a1, a2, a3, a4)
    
def run(a, a1, a2, a3, a4):
    run_online(a, a1, a2, a3, a4)

def print_to_csv(contentSimilarity, collaborativeSimilarity, userbasedSimilarity, slimSimilarity, graphSimilarity, a, a1, a2, a3, a4):
    file=open("hybrid-submission.csv",'a')
    file.write("playlist_id,track_ids"+"\n")
    recommender = Recommender(contentSimilarity, collaborativeSimilarity, userbasedSimilarity, slimSimilarity, graphSimilarity, a, a1, a2, a3, a4, b, "HYBRID")
    for playlist in b.get_ordered_target_playlists():
        s = str(recommender.recommend(playlist))
        s = s[1:len(s)-1]
        file.write(str(playlist)+","+s+"\n")
    for playlist in b.get_unordered_target_playlists():
        s = str(recommender.recommend(playlist))
        s = s[1:len(s)-1]
        file.write(str(playlist)+","+s+"\n")

In [6]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
b, ICM, URM_train, URM_train_transpose = start(False, -1)
recommender = SLIM_BPR_Cython(URM_train)
recommender.fit(epochs=75, batch_size=1000, lambda_i = 0.001, lambda_j = 0.001, sgd_mode='adagrad', learning_rate=0.02)
sim = recommender.W_sparse
scipy.sparse.save_npz('input/similarity/slimSIMOnline.npz', sim)

SLIM_BPR_Cython: Estimated memory required for similarity matrix of 20635 items is 1703.21 MB
Compiling in Cython
Compiled module saved in subfolder: /SLIM_BPR/Cython
Compilation Complete
Processed 1211791 ( 100.00% ) in 4.75 seconds. BPR loss is 7.73E-01. Sample per second: 255200
SLIM_BPR_Recommender: Epoch 1 of 75. Elapsed time 0.20 min
Processed 1211791 ( 100.00% ) in 4.62 seconds. BPR loss is 1.78E+00. Sample per second: 262479
SLIM_BPR_Recommender: Epoch 2 of 75. Elapsed time 0.27 min
Processed 1211791 ( 100.00% ) in 4.47 seconds. BPR loss is 2.50E+00. Sample per second: 270956
SLIM_BPR_Recommender: Epoch 3 of 75. Elapsed time 0.33 min
Processed 1211791 ( 100.00% ) in 4.41 seconds. BPR loss is 3.07E+00. Sample per second: 274982
SLIM_BPR_Recommender: Epoch 4 of 75. Elapsed time 0.40 min
Processed 1211791 ( 100.00% ) in 4.26 seconds. BPR loss is 3.58E+00. Sample per second: 284158
SLIM_BPR_Recommender: Epoch 5 of 75. Elapsed time 0.46 min
Processed 1211791 ( 100.00% ) in 4.32 seco

SLIM_BPR_Recommender: Epoch 53 of 75. Elapsed time 3.66 min
Processed 1211791 ( 100.00% ) in 4.32 seconds. BPR loss is 1.19E+01. Sample per second: 280454
SLIM_BPR_Recommender: Epoch 54 of 75. Elapsed time 3.73 min
Processed 1211791 ( 100.00% ) in 4.89 seconds. BPR loss is 1.20E+01. Sample per second: 247668
SLIM_BPR_Recommender: Epoch 55 of 75. Elapsed time 3.81 min
Processed 1211791 ( 100.00% ) in 5.39 seconds. BPR loss is 1.20E+01. Sample per second: 224920
SLIM_BPR_Recommender: Epoch 56 of 75. Elapsed time 3.88 min
Processed 1211791 ( 100.00% ) in 4.89 seconds. BPR loss is 1.21E+01. Sample per second: 247950
SLIM_BPR_Recommender: Epoch 57 of 75. Elapsed time 3.96 min
Processed 1211791 ( 100.00% ) in 4.94 seconds. BPR loss is 1.22E+01. Sample per second: 245152
SLIM_BPR_Recommender: Epoch 58 of 75. Elapsed time 4.02 min
Processed 1211791 ( 100.00% ) in 4.77 seconds. BPR loss is 1.22E+01. Sample per second: 253936
SLIM_BPR_Recommender: Epoch 59 of 75. Elapsed time 4.09 min
Processed 

In [ ]:
b, ICM, URM_train, URM_train_transpose = start(False, -1)
contentSimilarity, collaborativeSimilarity, userbasedSimilarity, slimSimilarity, graphSimilarity = compute_similarity()
run(0.06, 0.04, 0.10, 0.32, 0.20)

Similarity column 20600 ( 100 % ), 2681.84 column/sec, elapsed time 0.13 min
Similarity column 17200 ( 83 % ), 571.77 column/sec, elapsed time 0.50 min
Similarity column 20600 ( 100 % ), 572.47 column/sec, elapsed time 0.60 min
Similarity column 17400 ( 34 % ), 577.79 column/sec, elapsed time 0.50 min
Similarity column 33500 ( 66 % ), 557.11 column/sec, elapsed time 1.00 min
Similarity column 50400 ( 100 % ), 565.80 column/sec, elapsed time 1.48 min
